# Data quality check

> This notebook was created as part of the examination requirements of the "Introduction to Digital Humanities" class offered by the Master of Digital Humanities programme in KU Leuven.

This notebook serves to document the data quality check process that I used to familiarize myself with the dataset and to uncover the underlying data quality issues. The findings of this notebook fuel the code that is written for the exploration, cleaning, analysis and visualization code.

## Setup

### Import the required packages

In [1]:
from pathlib import Path

import pandas as pd

### Load the data

In [2]:
import_fp = Path("../data/raw/cornelia-raw.csv")
dataset = pd.read_csv(import_fp, encoding = "utf-8", sep=";")

## Structural Exploration

Let's first check to see how big the dataset is and what the columns are.

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830 entries, 0 to 2829
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   source            2830 non-null   object 
 1   source_entry      2830 non-null   object 
 2   date_day          2186 non-null   float64
 3   date_month        2204 non-null   float64
 4   date_year         2830 non-null   int64  
 5   actor_id          2830 non-null   int64  
 6   actor_first_name  2815 non-null   object 
 7   actor_surname     2823 non-null   object 
 8   role              2830 non-null   object 
 9   status            2830 non-null   object 
 10  phase             2744 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 243.3+ KB


The dataset has 2830 rows and eleven columns in total. Right out some potential problems become apparent:

- Some crucial attributes of dataset entries contain null values: actor_first_name and actor_surname should ideally not contain null values.
- date_day and date_month incorrectly show up as "float64" data type. This is most likely because the NULL nalues change the data type.

Looking at a few of the entries might also prove interesting:

In [4]:
dataset.head(5)

,source,source_entry,date_day,date_month,date_year,actor_id,actor_first_name,actor_surname,role,status,phase
0,BRGA818,00001599A,NaN,NaN,1599,5912,Bartholomeus,Van der Meren,member,dean,start
1,BRGA818,00001599A,NaN,NaN,1599,5913,Jan,Van Ponberghe,member,dean,start
2,BRGA818,00001599A,NaN,NaN,1599,6930,Carel,Van den Steen,member,dean,start
3,BRGA818,00001599B,NaN,NaN,1599,5917,Antoon,Van Schelle,member,master,start
4,BRGA818,00001599C,NaN,NaN,1599,5936,Abraham,Gyoryns,member,meesterszoon,start


## Column Exploration

Now that we know the general structure of the dataset, we can look at individual columns for problems:

### "source" column

In [5]:
print(dataset["source"].nunique()) # Only one value, all entries come from the same source.

1


There doesn't seem to be much of a problem with the "source" column.

### "source_entry" column

In [6]:
print(dataset["source_entry"].nunique()) # This column has differing values unlike the 'source' column
dataset["source_entry"].value_counts() #To see the top values

1467


00001671A    4
14041620C    4
1656         4
00001695A    4
1668         4
            ..
1041649      1
1041664      1
10071607     1
1061676      1
00001705E    1
Name: source_entry, Length: 1467, dtype: int64

When we look at the occurence count of each unique value, we see a potential problem: not all unique value has the same format. Some numbers seem to follow the full notation, but the others are more shortened. Let's explore this further:

In [7]:
source_entry_lengths = (dataset["source_entry"]
                        .str.len()).value_counts()
source_entry_lengths

9    1323
8     991
7     330
4     168
5      14
6       4
Name: source_entry, dtype: int64

It becomes apparent that there are **six different ways of indicating the source entry of a Cornelia entry** when the entry length is considered. However, there aren't six different notations in total. Further exploration to come with the notations can be followed:

In [8]:
dataset["source_entry"].str.len() == 9

dataset.loc[dataset["source_entry"].str.len() == 9, ["source_entry"]]

,source_entry
0,00001599A
1,00001599A
2,00001599A
3,00001599B
4,00001599C
...,...
2822,31051617B
2823,31051640A
2824,31051640A
2825,31051640B


In [9]:
notation_examples = {}

for key in source_entry_lengths.keys():
    key_mask = dataset["source_entry"].str.len() == key
    subset = dataset.loc[key_mask, ["source_entry"]]
    example = subset.iloc[0]
    notation_examples[key] = example
    
print(notation_examples)

{9: source_entry    00001599A
Name: 0, dtype: object, 8: source_entry    10011605
Name: 1176, dtype: object, 7: source_entry    1021611
Name: 645, dtype: object, 4: source_entry    1601
Name: 27, dtype: object, 5: source_entry    11600
Name: 626, dtype: object, 6: source_entry    101664
Name: 640, dtype: object}


The theory that we formulated by looking into the .csv file was that there is one canon source_entry notation and all others are deviations. This is partially true. Once we look at the neighboring columns of the 'source_entry' column, we see that this column is actually informed by the values coming from the 'date_day', 'date_month' and 'date_year' columns.

### 'date_day' column

In [10]:
print(len(dataset["date_day"].value_counts())) # Seems correct
print(dataset["date_day"].max(), dataset["date_day"].min()) # Also seems correct

null_mask = dataset["date_day"].isnull()
null_subset = dataset.loc[null_mask, :]
print(len(null_subset))
null_subset


31
31.0 1.0
644


,source,source_entry,date_day,date_month,date_year,actor_id,actor_first_name,actor_surname,role,status,phase
0,BRGA818,00001599A,NaN,NaN,1599,5912,Bartholomeus,Van der Meren,member,dean,start
1,BRGA818,00001599A,NaN,NaN,1599,5913,Jan,Van Ponberghe,member,dean,start
2,BRGA818,00001599A,NaN,NaN,1599,6930,Carel,Van den Steen,member,dean,start
3,BRGA818,00001599B,NaN,NaN,1599,5917,Antoon,Van Schelle,member,master,start
4,BRGA818,00001599C,NaN,NaN,1599,5936,Abraham,Gyoryns,member,meesterszoon,start
...,...,...,...,...,...,...,...,...,...,...,...
639,BRGA818,71654,NaN,7.0,1654,3244,Gaspar,Van den Bemde,painter,master,start
640,BRGA818,101664,NaN,10.0,1664,5519,Jan,Volsom,painter,master,start
641,BRGA818,101690,NaN,10.0,1690,5697,Andries,Van den Hoecke,member,apprentice,start
642,BRGA818,101690,NaN,10.0,1690,5358,Thomas,Pins,member,leermeester,start


It appears that there are 644 rows that do not have a date of enrollment recorded.

In [11]:
null_subset["source_entry"].value_counts()

1656         4
1668         4
00001695A    4
00001702A    4
00001671A    4
            ..
00001696C    1
00001695G    1
00001695F    1
00001695E    1
111607       1
Name: source_entry, Length: 314, dtype: int64

There isn't a single source entry that can be tracked as the source of these null values.

### 'date_month' column

In [12]:
print(len(dataset["date_month"].value_counts())) # Seems correct
print(dataset["date_month"].max(), dataset["date_month"].min()) # Also seems correct

null_mask = dataset["date_month"].isnull()
null_subset = dataset.loc[null_mask, :]
print(len(null_subset))
null_subset

12
12.0 1.0
626


,source,source_entry,date_day,date_month,date_year,actor_id,actor_first_name,actor_surname,role,status,phase
0,BRGA818,00001599A,NaN,NaN,1599,5912,Bartholomeus,Van der Meren,member,dean,start
1,BRGA818,00001599A,NaN,NaN,1599,5913,Jan,Van Ponberghe,member,dean,start
2,BRGA818,00001599A,NaN,NaN,1599,6930,Carel,Van den Steen,member,dean,start
3,BRGA818,00001599B,NaN,NaN,1599,5917,Antoon,Van Schelle,member,master,start
4,BRGA818,00001599C,NaN,NaN,1599,5936,Abraham,Gyoryns,member,meesterszoon,start
...,...,...,...,...,...,...,...,...,...,...,...
621,BRGA818,00001706A,NaN,NaN,1706,5904,Carel Judocus,Raes,member,dean,start
622,BRGA818,00001706A,NaN,NaN,1706,5589,Christiaen,Crocx,member,dean,start
623,BRGA818,00001706B,NaN,NaN,1706,5909,Johannes,Der Coor,glazier,master,start
624,BRGA818,00001706C,NaN,NaN,1706,5910,Jacques,Moreau,glazier,master,start


In [13]:
null_subset["source_entry"].value_counts()

1668         4
00001695A    4
00001671A    4
1656         4
00001702A    4
            ..
00001695E    1
00001695D    1
00001695C    1
00001695B    1
00001706D    1
Name: source_entry, Length: 303, dtype: int64

Once again, there isn't a single source entry that can be tracked as the source of these null values.

### Extra: 'date_day' and 'date_month' together

In [14]:
masks = {"both": (dataset["date_day"].isnull() & dataset["date_month"].isnull()),
         "day_only": (~(dataset["date_day"].isnull()) & dataset["date_month"].isnull()),
         "month_only":(dataset["date_day"].isnull() & (~(dataset["date_month"].isnull())))}

for key, mask in masks.items():
    subset = dataset.loc[mask, :]
    print(len(subset))
    

626
0
18


Here's a curious case: there isn't a single case where the day is recorded but the month is not. However, there are 18 cases where the month is recorded and the day is not. For the majority of cases, both the day and the month is NOT recorded.

### 'date_year' column

In [15]:
print(len(dataset["date_year"].value_counts())) # Seems correct
print(dataset["date_year"].max(), dataset["date_year"].min()) # Also seems correct

dataset["date_year"].value_counts()

108
1706 1599


1613    91
1640    72
1614    71
1653    67
1620    57
        ..
1685     6
1687     6
1704     6
1706     6
1697     3
Name: date_year, Length: 108, dtype: int64

There doesn't appear to be any problem with the 'date_year' column.

###  'actor_id' column

In [16]:
print(dataset["actor_id"].nunique() == len(dataset["actor_id"]))
# Since the ledger is kept over time,
# some actors register after x years with diff. role.
print(dataset["actor_id"].nunique())

appearence_times = pd.DataFrame((dataset["actor_id"]
                                 .value_counts()
                                 .reset_index()
                                 .rename(mapper = {"index": "actor_id", "actor_id": "count"},
                                         axis = 1)
                                ))
appearence_times = appearence_times["count"].value_counts()
appearence_times # However most actors appears only once

False
1260


1     825
2     143
3      89
4      54
5      36
6      25
7      23
8      15
9      10
10      8
12      8
13      7
19      5
11      4
14      4
18      2
16      1
21      1
Name: count, dtype: int64

The 'actor_id' column seems to be devoit of any problems. But who's that person that appears 21 times?

In [17]:
top_mention_mask = dataset["actor_id"] == dataset["actor_id"].value_counts().keys()[0]
top_mention = dataset.loc[top_mention_mask, :]
top_mention

,source,source_entry,date_day,date_month,date_year,actor_id,actor_first_name,actor_surname,role,status,phase
117,BRGA818,1626,NaN,NaN,1626,2829,Antoon,Van Opstal,member,dean,start
144,BRGA818,1639,NaN,NaN,1639,2829,Antoon,Van Opstal,member,dean,start
220,BRGA818,00001653O,NaN,NaN,1653,2829,Antoon,Van Opstal,painter,leermeester,start
917,BRGA818,5061652,5.0,6.0,1652,2829,Antoon,Van Opstal,member,leermeester,start
986,BRGA818,6081631,6.0,8.0,1631,2829,Antoon,Van Opstal,member,leermeester,start
996,BRGA818,7021629,7.0,2.0,1629,2829,Antoon,Van Opstal,painter,leermeester,start
1051,BRGA818,8021634,8.0,2.0,1634,2829,Antoon,Van Opstal,member,leermeester,start
1346,BRGA818,12031641B,12.0,3.0,1641,2829,Antoon,Van Opstal,painter,leermeester,former
1512,BRGA818,13111646C,13.0,11.0,1646,2829,Antoon,Van Opstal,member,leermeester,start
1580,BRGA818,14101625B,14.0,10.0,1625,2829,Antoon,Van Opstal,member,leermeester,start


### 'actor_first_name' column

In [18]:
print(dataset["actor_first_name"].value_counts()) # There are 215 unique names


null_mask = dataset["actor_first_name"].isnull()
null_subset = dataset.loc[null_mask, :]
print(len(null_subset)) # Only 15 missing names
null_subset

Jan                   380
Peter                 209
Francois              163
Antoon                111
Willem                 96
                     ... 
Germeyn                 1
Claude                  1
Joos Baptist            1
[widow of] Roeland      1
Dermitius               1
Name: actor_first_name, Length: 215, dtype: int64
15


,source,source_entry,date_day,date_month,date_year,actor_id,actor_first_name,actor_surname,role,status,phase
221,BRGA818,00001653P,NaN,NaN,1653,3243,NaN,Leyniers,glazier,apprentice,start
406,BRGA818,00001686E,NaN,NaN,1686,5638,NaN,Cotina,painter,apprentice,start
408,BRGA818,00001686F,NaN,NaN,1686,5639,NaN,Van der Borcht,painter,apprentice,start
522,BRGA818,00001698Q,NaN,NaN,1698,5796,NaN,Roemart,painter,recognitie,start
549,BRGA818,00001699O,NaN,NaN,1699,5824,NaN,Backer,member,leermeester,start
567,BRGA818,00001701E,NaN,NaN,1701,5839,NaN,Van der Cappen,painter,recognitie,start
575,BRGA818,00001701J,NaN,NaN,1701,5846,NaN,Hermans,glazier,apprentice,start
577,BRGA818,00001701K,NaN,NaN,1701,5850,NaN,Snaghels,painter,apprentice,start
586,BRGA818,00001702C,NaN,NaN,1702,5857,NaN,Crick,glazier,master,start
765,BRGA818,03071690C,3.0,7.0,1690,5692,NaN,Horick,member,apprentice,start


There are 15 entries in total who do not have a first name. Perhaps they can be filled in by looking at the entries with the same last name & same role? Also, this is not the true number of anonymous entities.

In [19]:
anonymous_mask = dataset["actor_first_name"] == "[anonymous]"
anonymous_subset = dataset.loc[anonymous_mask, :]
anonymous_subset #Two entries

null_mask = dataset["actor_first_name"].isnull()
totally_anonymous_mask = null_mask | anonymous_mask
totally_anonymous_subset = dataset.loc[totally_anonymous_mask, :]
print(len(totally_anonymous_subset)) # 17 missing names in total
totally_anonymous_subset



17


,source,source_entry,date_day,date_month,date_year,actor_id,actor_first_name,actor_surname,role,status,phase
221,BRGA818,00001653P,NaN,NaN,1653,3243,NaN,Leyniers,glazier,apprentice,start
406,BRGA818,00001686E,NaN,NaN,1686,5638,NaN,Cotina,painter,apprentice,start
408,BRGA818,00001686F,NaN,NaN,1686,5639,NaN,Van der Borcht,painter,apprentice,start
436,BRGA818,00001689D,NaN,NaN,1689,5672,[anonymous],NaN,painter,apprentice,start
467,BRGA818,00001694F,NaN,NaN,1694,5742,[anonymous],NaN,member,apprentice,start
522,BRGA818,00001698Q,NaN,NaN,1698,5796,NaN,Roemart,painter,recognitie,start
549,BRGA818,00001699O,NaN,NaN,1699,5824,NaN,Backer,member,leermeester,start
567,BRGA818,00001701E,NaN,NaN,1701,5839,NaN,Van der Cappen,painter,recognitie,start
575,BRGA818,00001701J,NaN,NaN,1701,5846,NaN,Hermans,glazier,apprentice,start
577,BRGA818,00001701K,NaN,NaN,1701,5850,NaN,Snaghels,painter,apprentice,start


### 'actor_surname' column

In [20]:
print(dataset["actor_surname"].value_counts()) # There are 864 unique surnames


null_mask = dataset["actor_surname"].isnull()
null_subset = dataset.loc[null_mask, :]
print(len(null_subset)) # Only 7 missing names
null_subset

Noveliers        40
Van Heil         39
Raes             34
Van den Steen    34
Sallaert         33
                 ..
Laseer            1
Bosdonck          1
Vylant            1
Van Ponberghe     1
Gieverckx         1
Name: actor_surname, Length: 864, dtype: int64
7


,source,source_entry,date_day,date_month,date_year,actor_id,actor_first_name,actor_surname,role,status,phase
436,BRGA818,00001689D,NaN,NaN,1689,5672,[anonymous],NaN,painter,apprentice,start
467,BRGA818,00001694F,NaN,NaN,1694,5742,[anonymous],NaN,member,apprentice,start
981,BRGA818,06071661A,6.0,7.0,1661,5705,Ambrosius,NaN,painter,apprentice,start
1490,BRGA818,13071686,13.0,7.0,1686,5617,Peter,NaN,member,apprentice,start
2105,BRGA818,21041610,21.0,4.0,1610,6480,Maerten,NaN,glazier,master,start
2334,BRGA818,24061654K,24.0,6.0,1654,3262,Maerten,NaN,member,leermeester,start
2638,BRGA818,28051683,28.0,5.0,1683,9053,Joos,NaN,glazier,apprentice,start


In [21]:

anonymous_mask = dataset["actor_surname"] == "[anonymous]"
anonymous_subset = dataset.loc[anonymous_mask, :]
len(anonymous_subset) # No "anonymous equivalent for surnames"

0

### Extra: totally anonymous actors

In [22]:
totally_anonymous_mask = ((dataset["actor_first_name"].isnull() & dataset["actor_first_name"] == "anonymous")
                          & (dataset["actor_surname"].isnull()))

totally_anonymous_subset = dataset.loc[totally_anonymous_mask, :]
totally_anonymous_subset

,source,source_entry,date_day,date_month,date_year,actor_id,actor_first_name,actor_surname,role,status,phase


There aren't any actors who are "totally anonymous". All actors have either a name or a surname.

### Extra: checking for ID - Name permenance

In [23]:
#Create a subset of actor_id's and full_name
id_names = dataset.loc[:, ["actor_id","actor_first_name", "actor_surname"]]
id_names["full_name"] = id_names["actor_first_name"] + " " + id_names["actor_surname"]
id_names = id_names.drop(labels = ["actor_first_name", "actor_surname"], axis = 1)

canon_names = {}

for actor_id, full_name in id_names.values:
    if actor_id not in canon_names:
        canon_names[actor_id] = []
    canon_names[actor_id].append(full_name)
    
for actor_id, names in canon_names.items():
    canon_names[actor_id] = set(names)
    if len(canon_names[actor_id]) > 2:
        print("Problem here!")

All ID's are bound to a single name only.

### 'role' column

In [24]:
print(dataset["role"].nunique()) # 10 unique values
print(sum(dataset["role"].isnull())) #no null values
dataset["role"].value_counts()

10
0


member            1673
painter            698
glazier            304
goudslager         115
vergulder           28
gelaesschryver       5
glasschilder         3
plaatslager          2
goldsmith            1
apotheker            1
Name: role, dtype: int64

###  'status' column

In [25]:
print(dataset["status"].nunique()) # 10 unique values
print(sum(dataset["status"].isnull())) #no null values
dataset["status"].value_counts()

8
0


leermeester     955
apprentice      919
master          473
dean            326
meesterszoon    103
recognitie       40
cortosie          7
ouderman          7
Name: status, dtype: int64

### 'phase' column

In [26]:
print(dataset["phase"].nunique()) # 10 unique values
print(sum(dataset["phase"].isnull())) #no null values
dataset["phase"].value_counts()

2
86


start     2485
former     259
Name: phase, dtype: int64

## Result

As a result, the following problems were noted:

**source**: no problems were noted

**source_entry**:
 - The source entry notations are not human readable
 - The source entry format is inconsistent: the source entry is influenced by date_day date_month and date_year columns. However, not all source entries are in this format. Sometimes the date_day and the date_month notation is missing.

**date_day**:
 - Has null values

**date_month**:
 - Has null values

**date_year**: no problems were noted

**actor_id**: no problems were noted

**actor_first_name**:
 - Has null values
 - Null values indicated with both "anonymous" and "NaN" and NN

**actor_surname**:
 - Has null values
 
**role**:
 - Some roles are noted only as "members"
 - Roles are recorded in both English and Dutch.

**status**:
- Roles are recorded in both English and Dutch.

**phase**:
 - Has null values